# 1. Libraries

In [1]:
### Computational Libraries
import numpy as np
import pandas as pd

### HTML Servlet related
import requests
import markdown2
from xml.dom.minidom import parseString


### News Summary Library
import newspaper

### NLTK
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords

### String Manipulation
from collections import defaultdict
from string import punctuation
from heapq import nlargest

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#### From Markdown to HTML

from webbrowser import open_new_tab


### Local HTML to PDF

import pdfcrowd
import sys

## Date for convention

from datetime import date
import calendar

# 2. Defined Classes

## 2.1 Class 'FrequencySummarizer' for Summarizing a Text

In [2]:
class FrequencySummarizer:
  def __init__(self, min_cut=0.5, max_cut=0.95):
    """ Words that have a frequency term lower than min_cut or higer than max_cut will be ignored. """
    self._min_cut = min_cut
    self._max_cut = max_cut 
    self._stopwords = set(stopwords.words('english') + list(punctuation))

  def _compute_frequencies(self, word_sent):
    """ 
      Input: a list of sentences already tokenized.
      Output:  a dictionary where freq[w] is the frequency of w.
    """
    freq = defaultdict(int)
    for s in word_sent:
      for word in s:
        if word not in self._stopwords:
          freq[word] += 1
    # frequencies normalization and fitering
    m = float(max(freq.values()))
    for w in list(freq):
      freq[w] = freq[w]/m
      if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
        del freq[w]
    return freq

  def summarize(self, text, n):
    """"
    Return a list of n sentences which represent the summary of text.
    """
    sents = sent_tokenize(text)
    assert n <= len(sents)
    word_sent = [word_tokenize(s.lower()) for s in sents]
    self._freq = self._compute_frequencies(word_sent)
    ranking = defaultdict(int)
    for i,sent in enumerate(word_sent):
      for w in sent:
        if w in self._freq:
          ranking[i] += self._freq[w]
    sents_idx = self._rank(ranking, n)    
    return [sents[j] for j in sents_idx]

  def _rank(self, ranking, n):
    """ return the first n sentences with highest ranking """
    return nlargest(n, ranking, key=ranking.get)

# 3. Web Search

*  The Data will be collected from the following News Sources only via Google-News:
    - goal.com  [HomePage](https://www.goal.com/en)
    - skysports.com  [HomePage](https://www.skysports.com/football)
    - espn.in  [HomePage](http://www.espn.in/football/?src=com)
    - theguardian.com  [HomePage](https://www.theguardian.com/football)


*  The News window is set to past 7 days.

- This is not an exhaustive list. You can customize this list by adding/removing websites. You can also filter particluar websited that you don't want in your search list.

- News wrt **Liverpool** and **English Premier League** will be fetched.
- News wrt **Liverpool** and **Champions League** will also be fetched separately.
- Then, I will combine both the News sources to only look for unique News.

# 4. News Compilation

In [2]:
urls = 0    ####Initialisation of the urls variable

sites = ['goal.com', 'skysports.com', 'espn.in', 'theguardian.com/football']    #### List to store the Websites

## 4.1 Function to fetch the URL and Title.

In [3]:
def URL_Decorator( func):
    """
    *  The function searches for the news as entered in the term 'search_string'
    *  The function also returns the title of the News for the fetched URL - retured by 'return_title' """
    def wrapper(string, days, urls):
        """ A Function Wrapper """
        term, count = func(string,days, urls)
        results = []
        results_title = []
        results_url = []
        obj = parseString( requests.get('https://news.google.com/rss/search?q={0}&hl=en-IN&gl=IN&ceid=IN:en'.format(term)).text )
        items = obj.getElementsByTagName('item') # Get each item
        for item in items[:count]:
            t,l = '', ''
            for node in item.childNodes:
                if node.nodeName == 'title':
                    t = node.childNodes[0].data
                elif node.nodeName == 'link':
                    l = node.childNodes[0].data
            results.append( (t,l) )
            results_title.append(t)
            results_url.append(l)
        return results_title, results_url
        
    return wrapper
    

@URL_Decorator
def search_strings(search_string, days, url):
    """
    A Function to write automated script as an input for 'get_google_new_results_test' function.
    
    search_string = Value that we need to search
    days = No of Days in the past
    urls = No of URL's to fetch
    """
    urls = int(url)
    for site in sites:
        if site == sites[0]:
            string = str('{0} {1}').format(search_string, site)
        elif site != sites[-1]:
            string = str(string) + str(' OR ') + str('{0}').format(site)
        else:
            string = string + str(' when:{0}d').format(days)
    return string, urls

# 7. Scraping Data into Markdown File

In [5]:
## Defining the Class Object ##
fs = FrequencySummarizer()

In [4]:
### Adding Naming Convension - based on Date
today_date = date.today()
today_date

day_file_name = str(today_date.day)+'/'+today_date.strftime('%m')+'/'+str(today_date.year)   ## In case Publish Date is absent
file2 = str(today_date.day)+'_'+today_date.strftime('%m')+'_'+str(today_date.year)
file_name = file2 + '_Report.pdf'

final_file_name = str(file_name)

In [5]:
%%capture output

### Setting search term
search_term = ['Liverpool AND English Premier League', 'Liverpool AND Champions League']
headings = ['English Premier League', 'Champions League']    #### Setting Headings

total_urls = []

for term, heading in zip(search_term,headings):
    
    title, urls = search_strings(('{0}').format(term), 20, 10)
    urls =  list(set(urls)^set(list(set(urls).intersection(total_urls))))  ## Only Unique URL's with respect to that fruit
    total_urls.extend(urls)
    
    print('<center><h3>' + str('{0}  Overview').format(str(heading))+'</h3></center>')

    for url in urls:
        try:
            common_stop_words = stopwords.words('english')  # Variable to store common English Stop words
            article = newspaper.Article(url)
            article.download()
            article.parse()
            article.nlp()

            title_url = article.title   # News Title
            text_text = article.text    # News Text
            title_genre =  article.keywords[0:5]
            #words = [ss('english').stem(word) for word in title_genre if word not in common_stop_words]
            words = [word for word in title_genre if word not in common_stop_words]

            summary = sent_tokenize(article.summary)
                
        except:
            continue

        ####################### Publish Date ####################################
        
        try:
            if(int(article.publish_date.month)>10):

                publish_date = str(article.publish_date.day)+'/' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
                publish_date = str(publish_date)
            else:
                publish_date = str(article.publish_date.day)+'/0' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
                publish_date = str(publish_date)
        except AttributeError:
            publish_date = str(day_file_name)
        ####################### Publish Date Ends ###############################


        ######################### Title #########################################
        print('<table>')

        print('<tr>')
        print('<th>')

        #print('[' + str(title_url)+ ']' + '('+ str(url) + ')' )
        print('<a href = "{0} ">'.format(url)+ str(title_url) +'</a>')
        #print(Date {0}.format(publish_date))

        #print('</div>')

        print('</th>')
        print('</tr>')
        print('</table>')              
            #print('----------------------------------')
        print('\n')
        print('\n')
        print('<article><strong>'+format(publish_date) + '</article>')
        
        ######################### Title Ends ######################################
        
        #print('\n {0}'.format(title_url))
        
        
        try:
            print(' ')
            print('<article>' + '<b style= "color:grey">Keywords:</b> <i>' + ' | '.join(words) + '</i> </article>')
            print('    ')
            print('------------------------------------------------------------------')
            """ Printing the Summary as bullet points."""
            for s in summary:
                print('* ', s)
            print('    ')
            
            print('    ')
            
            print('------------------------------------------------------------------')
            print('    ')
            
            print('------------------------------------------------------------------')
            print('    ')
        except AssertionError:
            continue
            
del total_urls

In [6]:
### Getting Text of the Output
Summary = output.stdout

In [7]:
## Direct Markdown conversion of Summary in 'test_summary'
test_summary = Markdown(data= Summary)
test_summary

<center><h3>English Premier League  Overview</h3></center>
<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/lebron-james-and-liverpool-the-la-lakers-stars-premier/l35fbosbgb3w1ep4cfs1totd1 ">LeBron James and Liverpool: The LA Lakers star’s Premier League investment explained</a>
</th>
</tr>
</table>




<article><strong>27/03/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>stars | explained | stake | business | premier</i> </article>
    
------------------------------------------------------------------
*  LeBron James and Liverpool: The LA Lakers star’s Premier League investment explainedA sensation on the court, 'King James' has also demonstrated that he has a nose for business and his empire spans sportsLos Angeles Lakers power forward LeBron James is one of the biggest stars in sport, but he is not just famed for his prowess on the basketball court.
*  When did LeBron James get involved with Liverpool?
*  Part of the deal with FSG was that he also acquired a minority stake in the Reds.
*  How much is LeBron James' stake in Liverpool worth?
*  Does LeBron James go to Liverpool games?
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/mane-liverpool-must-win-every-game-to-seal-premier-league/7rosgarfbri41prfb2nm8dfu4 ">Liverpool news: Reds must win every game to seal Premier League title - Sadio Mane</a>
</th>
</tr>
</table>




<article><strong>1/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>games | seal | win | mane | premier</i> </article>
    
------------------------------------------------------------------
*  Mane: Liverpool must win every game to seal Premier League titleThe Reds maintained their lead at the top of the table with victory over Tottenham, and their forward feels they cannot afford to drop any pointsSadio Mane believes will need to win all of their remaining games to have any chance of winning the Premier League.
*  The Reds kept their title hopes alive courtesy of a dramatic 2-1 victory over Tottenham at Anfield on Sunday.
*  And with six league games remaining, starting at on Friday night, Mane believes they will need to take maximum points in order to be champions.
*  “To be honest I think we need to win all of the games,” he said.
*  “Now we are back and we start with a win, I think it's very important.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/salah-nets-50th-premier-league-goal-for-liverpool-in-record/1hjzo62tbsadb15n056ih1cqxt ">Liverpool News: Mohamed Salah scores 50th Premier League goal for club in record time</a>
</th>
</tr>
</table>




<article><strong>5/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>goal | scores | record | premier | league</i> </article>
    
------------------------------------------------------------------
*  Salah nets 50th Premier League goal for Liverpool in record timeThe Reds Egyptian forward netted his half-century of goals for the club faster than anyone else, beating former Anfield star TorresMohamed Salah became the fastest player to reach 50 Premier League goals for as he helped fire the Reds back to the top of the Premier League on Friday.
*  💫 RECORD BREAKER 💫⚽ Games taken to net 50 @premierleague goals for the Reds…6⃣9⃣ Salah7⃣2⃣ Torres8⃣6⃣ Suarez8⃣8⃣ FowlerBrilliant from @MoSalah .
*  💪🔴 pic.twitter.com/1k89J6i8xT — Liverpool FC (@LFC) April 5, 2019Luis Suarez, the star of Liverpool's last serious title challenge in 2013-14, took 86 matches to score 50 Premier League goals for the club, while Robbie Fowler needed 88 games.
*  Earlier this year Salah became the fourth fastest player to 50 Premier League goals overall.
*  Liverpool had trailed to Shane Long's early opener before Naby Keita equalised with his first Premier League goal.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/training-with-juventus-is-harder-than-liverpool-emre-can/1m8e9cjsce5h11ktu67rk95mu3 ">Liverpool news: Training with Juventus more difficult than under Jurgen Klopp - Emre Can</a>
</th>
</tr>
</table>




<article><strong>10/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>training | appearances | klopp | decision | difficult</i> </article>
    
------------------------------------------------------------------
*  Training with Juventus is harder than Liverpool – Emre CanThe Germany midfielder says he has found difficulties with the Italian club while reflecting on his time in EnglandEmre Can says that he has found training with even more difficult than with despite having joked about how hard training is under Jurgen Klopp.
*  Can spent four seasons at Anfield after moving from in 2013, making 166 total appearances for the Premier League outfit.
*  But he departed the Premier League for after the 2017-18 season, joining up with Juventus on a free transfer.
*  “I already complained to Jurgen Klopp about the hard training in Liverpool!
*  "He also knew that it was not a decision against him or against Liverpool, but for a new challenge at Juventus.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/im-praying-that-they-do-it-wilson-backs-liverpool-title/1vsc4klkjh5171id2ohxgzxel7 ">Liverpool news: 'I'm praying that they do it' - Harry Wilson back Reds' title challenge</a>
</th>
</tr>
</table>




<article><strong>10/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>im | challenge | premier | league | sgorio</i> </article>
    
------------------------------------------------------------------
*  'I'm praying that they do it' - Wilson backs Liverpool title challengeThe Wales international is hoping that the Reds can go on to win the league this seasonHarry Wilson, on loan at from , says he is "praying" that his parent club can "get the job done" and clinch a first-ever Premier League title.
*  In his absence, Liverpool have enjoyed a stellar season and are matching reigning champions stride for stride in a thrilling Premier League title race.
*  Despite being away from the club, Wilson admits that he has enjoyed watching the Reds perform so admirably.
*  Liverpool at the top end of the table and I’m here praying that they do it."
*  Mwy o Sgorio are also giving away a Wales shirt signed by Harry Wilson – visit @sgorio on Twitter and Facebook to enter.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/man-city-stronger-liverpool-laporte/1rsy37jzjzo8911p56ollhsd1y ">Man City news: 'We are stronger than Liverpool' - Aymeric Laporte</a>
</th>
</tr>
</table>




<article><strong>7/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>win | france | city | team | league</i> </article>
    
------------------------------------------------------------------
*  'Man City are stronger than Liverpool' - LaporteThe 24-year-old is determined to win every competition with Pep Guardiola's side and opened up on his situation with the France national teamdefender Aymeric Laporte says his side are stronger than Premier League title rivals and is aiming to win every trophy he can with the club.
*  City are two points behind Liverpool in the English top flight and have played a game less as the campaign enters its final weeks.
*  And Laporte is confident his side can make the campaign a resounding success, hoping they end the season "in style".
*  He added: "Are we stronger than Liverpool?
*  Despite his integral role in the reigning Premier League champions' back four, Laporte is yet to receive his first cap for .
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-gb/news/liverpool-motivated-to-win-premier-league-says-sadio-mane/njdb4txacpjczvdbxz4uu8co ">Liverpool motivated to win Premier League, says Sadio Mane</a>
</th>
</tr>
</table>




<article><strong>5/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>games | urged | going | try | win</i> </article>
    
------------------------------------------------------------------
*  Liverpool motivated to win Premier League, says Sadio ManeThe Senegal international has urged his side to win their last six games of the season to stand a chance of ending their title droughtSadio Mane believes are well motivated to clinch the Premier League trophy this season.
*  The Anfield outfit have failed to win a title since claiming the League Cup in 2012 and last won the English top-flight trophy in 1990.
*  “I’m confident we can do it, we’re going to try to do everything to win all six games.
*  “We are in two different competitions, the Premier League and Champions League, and we want to win both of them.
*  Our main target is to win the league and the Champions League because we have the quality in the team and the staff, the club is amazing.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/an-embarrassment-chelsea-condemn-fans-after-video-emerges-of/1bbqbkthq3qxp1e4m7flr9pb5k ">Racism in football: Chelsea condemn fans after video emerges of Mohamed Salah chants</a>
</th>
</tr>
</table>




<article><strong>11/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>video | chelsea | racism | chants | football</i> </article>
    
------------------------------------------------------------------
*  'An embarrassment' - Chelsea condemn fans after video emerges of racist Salah chantsBlues supporters have once again caused controversy as footage of them singing offensive songs in Prague was circulated onlinehave condemned a video showing what is claimed to be Blues fans in Prague calling star Mohamed Salah "a bomber".
*  "Such individuals are an embarrassment to the vast majority of Chelsea supporters who won’t tolerate them in their club."
*  Chelsea fans have been under scrutiny after Raheem Sterling was allegedly abused at Stamford Bridge in Chelsea's 2-0 win over in December.
*  UEFA also investigated accusations that Blues supporters were singing anti-Semitic songs relating to their rivalry with during their game against Vidi just a week after the Sterling incident.
*  We will be liaising with Chelsea Football Club to ensure those involved are identified and punished swiftly and effectively."
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/premier-leagues-top-six-is-like-playing-bayern-munich-12/1gk87uh6qteqd1lgzlil4gxnml ">Southampton vs Liverpool: Premier League's top six is like playing Bayern Munich 12 times! - Ralph Hasenhuttl</a>
</th>
</tr>
</table>




<article><strong>5/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>quality | times | munich | 12 | ralph</i> </article>
    
------------------------------------------------------------------
*  Premier League's top six is like playing Bayern Munich 12 times!
*  are next on the agenda, with Hasenhuttl telling Goal and Spox of the elite competition he now finds himself up against: “We have six teams at the level of Bayern Munich in the Premier League.
*  So, you could say we play 12 times against Bayern per season.
*  And then we are the , or of the Premier League.
*  That makes it a special challenge.”On the collective quality of the Premier League being above that in the , Hasenhuttl added: “The top clubs in the Premier League benefit very much from the fact that there are six teams on a very high level.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-ph/news/will-man-city-or-liverpool-win-the-premier-league-title-race/mh4fwnvyg9kh13flutri7bg3f ">Will Man City or Liverpool win the Premier League? Title race, fixtures & latest points</a>
</th>
</tr>
</table>




<article><strong>11/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>fixtures | win | premier | city | league</i> </article>
    
------------------------------------------------------------------
*  Will Man City or Liverpool win the Premier League?
*  Title race, fixtures & latest pointsThe race for the league title is heading right for the wire as Klopp and Guardiola's battle could continue until the final dayWith the Premier League season winding down to a close, and Man City have emerged as the frontrunners to lift the league title.
*  Goal takes a look at the league standings, key fixtures that could determine the title race for Liverpool and Man City, and more.
*  Date Match Competition April 14 Liverpool vs Chelsea Premier League April 17 vs Liverpool April 21 Cardiff vs Liverpool Premier League April 26 Liverpool vs Huddersfield Premier League May 5 Newcastle vs Liverpool Premier League May 12 Liverpool vs Premier LeagueLiverpool have the benefit in only contending for silverware in two competitions as they have a considerably lighter fixture list than Man City.
*  Date Match Competition April 14 vs Man City Premier League April 17 Man City vs Tottenham Champions League April 20 Man City vs Tottenham Premier League April 24 Man United vs Man City Premier League April 28 vs Man City Premier League May 4 Man City vs Leicester Premier League May 12 vs Man City Premier League May 18 Man City vsMan City still have the quadruple in sight as they look to contend for FA Cup, Premier League and Champions League victories following on from their win in February.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<center><h3>Champions League  Overview</h3></center>
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-sg/news/champions-league-2018-19-porto-liverpool-revenge/otrjn33kftlf1285ljd456vb9 ">UEFA Champions League 2018-19: Can Porto get their revenge on Liverpool?</a>
</th>
</tr>
</table>




<article><strong>7/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>revenge | win | league | uefa | 16</i> </article>
    
------------------------------------------------------------------
*  UEFA Champions League 2018-19: Can Porto get their revenge on Liverpool?
*  They might not have had the trickiest of opponents to navigate past in the Champions League group stage but have still done well to top a group also containing , and .
*  The Mali international has been Porto’s guiding beacon in the Champions League this season with six goals along with two assists and he has the capability to punish the Liverpool defence when the chance arises.
*  Like Liverpool, Porto have plenty of history when it comes to the Champions League and are no strangers to battling it out in the knock-out stages of the competition.
*  Revenge will be on the minds of the 28-time Primeira League champions and Liverpool should write them off at their own peril.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-in/news/champions-leagues-greatest-goalkeeper-casillas-showing-no/vqhzxnqvg3er17rvuimomj069 ">Liverpool v Porto: Iker Casillas showing no signs of hanging up his gloves</a>
</th>
</tr>
</table>




<article><strong>8/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>madrid | hanging | iker | casillas | team</i> </article>
    
------------------------------------------------------------------
*  Champions League’s greatest goalkeeper Casillas showing no signs of hanging up his glovesThe Champions League king is back on centre stage once again and the lights aren't ready to be turned off just yetThere is a reason why Iker Casillas is nicknamed 'San Iker' - which translated from Spanish means 'Saint Iker'.
*  Casillas' 57 clean sheets in 175 Champions League matches eclipses Buffon and Van der Sar (both 51) and he is the only player to have reached the knockout stages on 19 occasions.
*  Article continues belowPorto have a rich Champions League history themselves, winning two titles - including the 2003-04 edition under Jose Mourinho.
*  And with Real Madrid, PSG and already eliminated this time around, the Portuguese giants will believe they can go deep into the competition and cause a few surprises.
*  Thus, the stage is set for the ageless Casillas to once again show us why he is called ‘San Iker’.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/we-dominated-liverpool-for-110-minutes-maldini-reflects-on-cruel-/42bimr27gbj1kegqp2hgdtry ">Champions League news: 'We dominated Liverpool for 110 minutes' - Paolo Maldini reflects on 'cruel' 2005 Istanbul final</a>
</th>
</tr>
</table>




<article><strong>5/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>goal | dominated | penalty | maldini | minutes</i> </article>
    
------------------------------------------------------------------
*  'We dominated Liverpool for 110 minutes' - Maldini reflects on 'cruel' 2005 Champions League finalThe Italian legend says that the Rossoneri's famous defeat against the Reds was "cruel" and that it fuelled their desire for revenge two years laterPaolo Maldini insists that "dominated" the 2005 final against , despite losing the match after a dramatic penalty shootout.
*  Carlo Ancelotti's side raced into a 3-0 lead during a frenetic first half, with Maldini heading home the opening goal after just 50 seconds.
*  "Yes I think I watch the Istanbul game again for the first time after a year," he told Goal and DAZN.
*  Dudek was again the saviour in the shootout as he saved Milan's final penalty from Andriy Shevchenko and the Reds basked in their fifth European Cup victory.
*  A 2-1 victory in Athens exorcised the demons of Istanbul for Milan, as Maldini stated, before also confessing that he very nearly missed the 2007 final through injury.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-sg/news/champions-league-qf-highlights-liverpool-porto-man-city/mpol8fm9nxui16wxuw79yp0ml ">UEFA Champions League Quarter-finals: 1st Leg Highlights from Wednesday</a>
</th>
</tr>
</table>




<article><strong>10/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>leg | 1st | highlights | quarterfinal | win</i> </article>
    
------------------------------------------------------------------
*  UEFA Champions League Quarter-finals: 1st Leg Highlights from WednesdayHere are the highlights from the first leg of the quarter-final matches that unfolded on Wednesday..
*  The first legs of the quarter-final of the UEFA concluded on Wednesday with two cracking matches.
*  Watch the highlights from those games:1) 1 0: A second-half strike from Son Heung-min gave Tottenham a 1-0 win over Manchester City in the first leg of their Champions League quarter-final tie.
*  Sergio Aguero missed a first-half penalty and Harry Kane was forced off injured before Son's strike settled the first European match at the new Tottenham Hotspur Stadium on Tuesday.
*  2) 2 0: Naby Keita and Roberto Firmino were on target as Liverpool eased to a 2-0 win in the first leg of their Champions League quarter-final against Porto.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-sg/lists/can-moussa-marega-join-an-elite-champions-league-club-vs/h8mrfw6izlyf1ukpnq6r09b96 ">Can Moussa Marega join elite Champions League club vs. Liverpool?</a>
</th>
</tr>
</table>




<article><strong>9/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>marega | ucl | join | moussa | premier</i> </article>
    
------------------------------------------------------------------
*  Moussa Marega is on the brink of joining an elite group of players.
*  The Mali international has been in sublime form in Europe’s premier club competition, having netted in his last six successive matches in this season’s UCL.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-in/news/bayern-legend-matthaus-backs-liverpool-for-champions-league/h53pt2w442al13f4gza74yxw0 ">Champions League news: Bayern legend Lothar Matthaus backs Liverpool for European glory</a>
</th>
</tr>
</table>




<article><strong>3/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>teams | matthaus | glory | win | league</i> </article>
    
------------------------------------------------------------------
*  Bayern legend Matthaus backs Liverpool for Champions League gloryThe former Germany international is tipping the Reds to win the competition and says his former side will strengthen in the summerlegend Lothar Matthaus is tipping to win the after knocking out his former team.
*  He told Goal and SPOX: "Other countries have had dry stretches and our teams have met strong opponents in the round of 16.
*  "Last year Bayern had , this time Liverpool, my personal favourite for the title.
*  "Bayern will still have to sign one or two other players to be on par with the teams from , , and ," added the former World Player of the Year.
*  "Thorgan Hazard is a good player, but I do not know if he is good enough to help Bayern Munich.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-sg/news/who-has-won-the-most-champions-league-titles-finals-winners-team-/4mubtf8re0k1tmvl29etmc7u ">Who has won the most Champions League titles? Finals, winners & team records</a>
</th>
</tr>
</table>




<article><strong>9/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>times | winners | team | league | trophy</i> </article>
    
------------------------------------------------------------------
*  Who has won the most Champions League titles?
*  Who has won the most Champions League titles?
*  They are followed by Milan in second place, who have clinched seven Champions League titles.
*  are still seeking their first Champions League trophy since 1996, which would be their third victory.
*  Which teams get to keep the Champions League trophy?
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    
<table>
<tr>
<th>
<a href = "https://www.goal.com/en-in/lists/from-ajax-to-manchester-city-champions-league-power-rankings/1rpli0nhjpwty1lqjvmvqucuaq ">From Ajax to Manchester City: Champions League Power Rankings</a>
</th>
</tr>
</table>




<article><strong>8/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>rankings | wire | ajax | premier | city</i> </article>
    
------------------------------------------------------------------
*  Having made light work of Bayern in the round of 16, are paired with a side they will be expecting to swat away.
*  There is a steely determination about Liverpool this season – epitomised by Virgil van Dijk at the back, and they are no less ruthless in attack.
*  This team – beaten finalists last year – are experienced and know all about what it takes to succeed at this level.
*  If you asked any Liverpool fan if they’d prefer the Champions League or Premier League title, they would probably opt for the latter after a wait of 29 years.
*  However, they don’t have to make a decision just yet with the team in the shakeup for both.
    
    
------------------------------------------------------------------
    
------------------------------------------------------------------
    


### Conversion of Markdown to HTML

In [8]:
html = markdown2.markdown(str(Summary), html4tags=True, tab_width=5,  )

### Cleaning the HTML File

In [9]:
### Rectify Out of Bound Paras
html = html.replace("<p>","<ul><li><p>")

html = html.replace('</p>', '</p></li></ul>')

#### Rectify Headlines Link
html = html.replace("<ul><li><p><a href","<p><a href")

html = html.replace('/a></p></li></ul>', '/a></p>')

###  Rectify <centre>
html = html.replace("<ul><li><p><center></p></li></ul>","<p><center></p>")

html = html.replace('<ul><li><p></center></p></li></ul>', '<p></center></p>')


### Rectify *

html = html.replace("*","</li><li>")

### Rectify Image

html = html.replace("<li><img ","<lily imagine ")

#html = html.replace("</h1></center></p></li></ul>","</h1></center></p>")

## Rectify Title Bullet point

html = html.replace("<ul><li><p><center><h3>","<p><center><h3>")

html = html.replace("</h3></center></p></li></ul>","</h3></center></p>")

## Open in new tab

html = html.replace("<a href", '<a rel="noopener noreferrer" target="_blank" href')

### Checking the sample Webpage

In [10]:
filename = 'File.html'

f = open(filename,'w', encoding='utf-16')

wrapper = """<!DOCTYPE html>
<html lang="en">



<head>

<style>
.center {
  text-align: center;
  
}




</style>


<meta charset="ISO-8859-1" >
<div class="page center">
   <a href="https://www.liverpoolfc.com/welcome-to-liverpool-fc"><img class="auto-size__target postpone-load--fade-in widge-figure__image" data-src="https://e1.365dm.com/18/06/768x432/skysports-liverpool-fixtures_4325753.jpg?20180601143103" alt="" itemprop="image" src="https://e1.365dm.com/18/06/768x432/skysports-liverpool-fixtures_4325753.jpg?20180601143103" style="PADDING-RIGHT: 70px"></a>
</div>

<title>%s</title>

</head>

<body>
    
    %s
</body>



</html>"""

title = str('Liverpool News')

whole = wrapper % (title, html)

#whole = wrapper % (html)

f.write(whole)

f.close()

open_new_tab(filename)

True

## Conversion to PDF

In [11]:
try:
    # create the API client instance
    client = pdfcrowd.HtmlToPdfClient('<User_Name>', '<Secret_API_Key>')

    # run the conversion and write the result to a file
    client.convertFileToFile('File.html', final_file_name )
except pdfcrowd.Error as why:
    # report the error
    sys.stderr.write('Pdfcrowd Error: {}\n'.format(why))

    # handle the exception here or rethrow and handle it at a higher level
    raise